In [1]:
# LISBON GEOSPATIAL ANALYSIS
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import os
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# Set Plotly default theme
import plotly.io as pio
pio.templates.default = "plotly_white"

def normalize_freguesia(df, col_name):
    if col_name in df.columns:
        df['Freguesia_Norm'] = df[col_name].str.strip().str.upper()
    return df

def plot_choropleth(gdf, column, title, cmap='viridis', vmin=None, vmax=None, categorical=False):
    """
    Consistent plotting function using Plotly for interactivity.
    """
    # Check if column exists
    if column not in gdf.columns:
        print(f"Column {column} not found. Skipping.")
        return

    # Reproject to WGS84 for Mapbox
    gdf_4326 = gdf.to_crs("EPSG:4326")
    
    # Create Hover Data
    hover_data = {'Freguesia_Norm': True, column: True}
    
    # Handle Colormap mapping (Matplotlib names to Plotly names/lists)
    color_scale = cmap
    if cmap == 'RdBu': color_scale = 'RdBu'
    elif cmap == 'OrRd': color_scale = 'OrRd'
    elif cmap == 'YlGn': color_scale = 'YlGn'
    elif cmap == 'Purples': color_scale = 'Purples'
    elif cmap == 'Blues': color_scale = 'Blues'
    elif cmap == 'Greens': color_scale = 'Greens'
    elif cmap == 'magma': color_scale = 'Magma'
    elif cmap == 'viridis': color_scale = 'Viridis'
    
    # Determine range
    range_color = [vmin, vmax] if vmin is not None and vmax is not None else None

    if categorical:
        fig = px.choropleth_mapbox(
            gdf_4326,
            geojson=gdf_4326.geometry,
            locations=gdf_4326.index,
            color=column,
            hover_name='Freguesia_Norm',
            hover_data={column: True},
            title=title,
            mapbox_style="carto-positron",
            center={"lat": 38.7223, "lon": -9.1393},
            zoom=11,
            opacity=0.7,
            color_discrete_sequence=px.colors.qualitative.Bold
        )
    else:
        fig = px.choropleth_mapbox(
            gdf_4326,
            geojson=gdf_4326.geometry,
            locations=gdf_4326.index,
            color=column,
            hover_name='Freguesia_Norm',
            hover_data={column: True},
            title=title,
            mapbox_style="carto-positron",
            center={"lat": 38.7223, "lon": -9.1393},
            zoom=11,
            opacity=0.7,
            color_continuous_scale=color_scale,
            range_color=range_color
        )
        
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.show()


In [2]:
# 0. LOAD BASE GEOMETRY
from shapely.geometry import Polygon

freguesias_path = "../data/boundaries/lisboa_freguesias_oficial.geojson"
freguesias_gdf = gpd.read_file(freguesias_path).to_crs("EPSG:3763")
name_col = 'Des_Simpli' if 'Des_Simpli' in freguesias_gdf.columns else 'Freguesia'
freguesias_gdf = normalize_freguesia(freguesias_gdf, name_col)

# --- MANUAL CLIP WATER (Tagus River) ---
# Since we lack a precise land mask, we approximate the river boundary to fix densities.
# Coordinates are approximate trace of the coastline.
coast_points = [
    (-9.300, 38.690), # West limit
    (-9.235, 38.691), # Belém Tower area
    (-9.180, 38.695), # Alcântara
    (-9.150, 38.703), # Terreiro do Paço
    (-9.120, 38.710), # Santa Apolónia
    (-9.100, 38.735), # Beato/Marvila
    (-9.090, 38.750), # Braço de Prata
    (-9.085, 38.790), # Parque das Nações
    (-9.085, 38.850), # North limit (river side)
    (-8.900, 38.850), # East
    (-8.900, 38.500), # South East
    (-9.300, 38.500)  # South West
]

water_poly = Polygon(coast_points)
water_gdf = gpd.GeoDataFrame({'geometry': [water_poly]}, crs="EPSG:4326").to_crs("EPSG:3763")

# Clip (Difference)
freguesias_gdf = gpd.overlay(freguesias_gdf, water_gdf, how='difference')

# Recalculate Area
freguesias_gdf['Area_km2'] = freguesias_gdf.geometry.area / 10**6

# Master DataFrame for aggregations
master_stats = freguesias_gdf[['Freguesia_Norm', 'geometry', 'Area_km2', name_col]].copy()

print("Map clipped to remove water areas. Areas recalculated.")


Map clipped to remove water areas. Areas recalculated.


In [3]:
# 5. CULTURE
print("--- 5. Culture ---")
import os

# Load all culture files
culture_dir = "../data/culture"
culture_files = [f for f in os.listdir(culture_dir) if f.endswith('.json')]

culture_gdfs = []
for f in culture_files:
    path = os.path.join(culture_dir, f)
    try:
        # Try reading as GeoDataFrame first
        try:
            gdf_c = gpd.read_file(path)
        except:
            # Fallback for plain JSON with lat/lon
            df_c = pd.read_json(path)
            if 'lat' in df_c.columns and 'lon' in df_c.columns:
                gdf_c = gpd.GeoDataFrame(df_c, geometry=gpd.points_from_xy(df_c.lon, df_c.lat), crs="EPSG:4326")
            else:
                continue
        
        # Ensure CRS is EPSG:3763
        if gdf_c.crs is None:
            gdf_c.set_crs("EPSG:4326", inplace=True)
        gdf_c = gdf_c.to_crs("EPSG:3763")
        
        gdf_c['Type'] = f.replace('.json', '').replace('lisbon', '').strip()
        # Keep only necessary columns to avoid schema conflicts
        gdf_c = gdf_c[['geometry', 'Type']]
        culture_gdfs.append(gdf_c)
    except Exception as e:
        print(f"Error loading {f}: {e}")

if culture_gdfs:
    culture_gdf = pd.concat(culture_gdfs, ignore_index=True)
    
    # 5.1 Map of Cultural Installations
    culture_4326 = culture_gdf.to_crs("EPSG:4326")
    fig_cult = px.scatter_mapbox(
        culture_4326, lat=culture_4326.geometry.y, lon=culture_4326.geometry.x,
        color='Type', zoom=11, title="5.1 Cultural Installations Map",
        mapbox_style="carto-positron"
    )
    fig_cult.show()
    
    # 5.2 Density per Freguesia
    # Spatial join to count venues per freguesia
    cult_joined = gpd.sjoin(culture_gdf, freguesias_gdf, how="inner", predicate="within")
    cult_counts = cult_joined.groupby('Freguesia_Norm').size().reset_index(name='Culture_Count')
    
    master_stats = master_stats.merge(cult_counts, on='Freguesia_Norm', how='left').fillna(0)
    # Fix column name collision if re-running
    if 'Culture_Count_x' in master_stats.columns:
        master_stats['Culture_Count'] = master_stats['Culture_Count_x'] + master_stats['Culture_Count_y']
        master_stats.drop(columns=['Culture_Count_x', 'Culture_Count_y'], inplace=True)
        
    master_stats['Culture_Density'] = master_stats['Culture_Count'] / master_stats['Area_km2']
    
    plot_choropleth(master_stats, 'Culture_Density', "5.2 Density of Cultural Installations per km²", cmap='Purples')
else:
    print("No culture data loaded.")


--- 5. Culture ---
